In [1]:
import os 
current_dir = os.getcwd()
print(current_dir)
os.chdir("../")

d:\interview\End-to-End-DL\research


In [2]:
from cnnClassifier.utils import read_yaml , create_directories
from cnnClassifier.constants import  *
from dataclasses import dataclass
from pathlib import Path
import os
from cnnClassifier import logging
from dataclasses import dataclass
from pathlib import Path


# @dataclass(frozen=True)
# class PrepareBaseModelConfig:
#     root_dir: Path
#     base_model_path: Path
#     update_base_model_path: Path
#     params_image_size: list
#     params_learning_rate: float
#     params_include_top: bool
#     params_classes = int
#     params_weights: str


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    update_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int


class ConfigurationManager:
    def __init__(self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):
        self.config_file = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config_file.artifacts_root])

    def get_prepare_model_config(self) -> PrepareBaseModelConfig:
        config = self.config_file.prepare_base_model
        create_directories([config.root_dir])
        
        # prepare_base_model_config = PrepareBaseModelConfig(
        #     root_dir = Path(config.root_dir),
        #     base_model_path = Path(config.base_model_path),
        #     update_base_model_path = Path(config.update_base_model_path),
        #     params_image_size= self.params_file.IMAGE_SIZE,
        #     params_learning_rate= self.params_file.LEARNING_RATE,
        #     params_include_top= self.params_file.INCLUDE_TOP,
        #     params_classes = self.params_file.CLASSES,
        #     params_weights = self.params_file.WEIGHTS
        # )
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            update_base_model_path=Path(config.update_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )
        return prepare_base_model_config

In [3]:
import tensorflow as tf
class BaseModel: 
    def __init__(self, config:PrepareBaseModelConfig):
        self.config = config
        # self.base_model = None
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            include_top=self.config.params_include_top,
            weights=self.config.params_weights,
            input_shape=self.config.params_image_size
        )
        self.save_model(path = self.config.base_model_path , model = self.model)
    @staticmethod
    def _prepare_full_model(model , classes , freeze_all , freeze_till , learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False

        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:freeze_till]:
                model.trainable = False
        flatten_in = tf.keras.layers.Flatten()(model.output)    
        prediction = tf.keras.layers.Dense(classes, activation='softmax')(flatten_in)                                                                                                                                                                                                    
        full_model = tf.keras.models.Model(inputs = model.input, outputs = prediction)
        full_model.compile(
            loss = tf.keras.losses.categorical_crossentropy,
            optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
            metrics = ['accuracy']
        )
        full_model.summary()
        return full_model
    
    @staticmethod
    def save_model(path:Path, model:tf.keras.models.Model):
        model.save(path)
        
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
        model = self.model,
        classes = self.config.params_classes,
        freeze_all = True,
        freeze_till = None,
        learning_rate = self.config.params_learning_rate
        )
        self.save_model(path = self.config.update_base_model_path , model = self.full_model)

[2024-01-22 21:34:41,191: INFO: utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2024-01-22 21:34:41,191: INFO: utils: NumExpr defaulting to 8 threads.]


In [4]:

try:
    config_manager = ConfigurationManager()
    base_model_config = config_manager.get_prepare_model_config()
    base_model = BaseModel(config = base_model_config)
    base_model.get_base_model()
    base_model.update_base_model()
except Exception as e:
    print(e)
    logging(e)
    

[2024-01-22 21:34:43,263: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-22 21:34:43,279: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-22 21:34:43,287: INFO: common: created directory at: artifacts]
[2024-01-22 21:34:43,290: INFO: common: created directory at: artifacts/prepare_base_model]


[2024-01-22 21:34:50,293: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                          